In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")
example = pd.read_csv("/kaggle/input/titanic/gender_submission.csv")

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [4]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### Removing irrelevant columns

In [5]:
train_v1 = train.drop(columns = ['Name', 'Ticket', 'Cabin'])

train_v1.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [6]:
train_v1['Embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)

### Changing categoric to numeric

In [7]:
sex_change = {'male': 1, 'female': 0}

train_v1['Sex'] = train.Sex.map(sex_change)

embark_change = {'S': 1, 'C': 2, 'Q': 3}

train_v1['Embarked'] = train.Embarked.map(embark_change)

train_v1.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,1.0
1,2,1,1,0,38.0,1,0,71.2833,2.0
2,3,1,3,0,26.0,0,0,7.9250,1.0
3,4,1,1,0,35.0,1,0,53.1000,1.0
4,5,0,3,1,35.0,0,0,8.0500,1.0


In [8]:
train_v1.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         2
dtype: int64

In [9]:
train_v1.fillna(train_v1.median(), inplace=True)

In [10]:
train_v1.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [11]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [12]:
test_v1 = test.drop(columns = ['Name', 'Ticket', 'Cabin'])
test_v1['Sex'] = train.Sex.map(sex_change)
test_v1['Embarked'] = train.Embarked.map(embark_change)

test_v1.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1,34.5,0,0,7.8292,1.0
1,893,3,0,47.0,1,0,7.0000,2.0
2,894,2,0,62.0,0,0,9.6875,1.0
3,895,3,0,27.0,0,0,8.6625,1.0
4,896,3,1,22.0,1,1,12.2875,1.0


In [13]:
test_v1.fillna(test_v1.median(), inplace=True)
test_v1.isnull().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

## Iniciando modelagem

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from numpy import random

SEED = 666
random.seed(SEED)

x = train_v1.drop(columns = ['Survived'])
y = train_v1['Survived']

train_x, test_x, train_y, test_y = train_test_split(x, y, 
                                                        train_size=0.7, random_state=SEED)

model = RandomForestClassifier(n_estimators=200)
model.fit(train_x, train_y)
print("A acurácia foi de %.2f%%" % (model.score(test_x, test_y) * 100))

A acurácia foi de 83.21%


In [15]:
from sklearn.dummy import DummyClassifier

SEED = 123321
random.seed(SEED)

classificador_bobo = DummyClassifier(strategy='most_frequent')

classificador_bobo.fit(train_x, train_y)
print("A acurácia foi de %.2f%%" % (classificador_bobo.score(test_x, test_y) * 100))

A acurácia foi de 59.33%


## Aplicando ao 'test' real para submissão

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from numpy import random

SEED = 666
random.seed(SEED)

x = train_v1.drop(columns = ['Survived'])
y = train_v1['Survived']

model = RandomForestClassifier(n_estimators=200)
model.fit(x, y)

predictions = model.predict(test_v1)

output = pd.DataFrame({'PassengerId': test_v1.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
